# 수비 이그노벨상 계산기

K리그 선수들의 "수비 이그노벨상"을 계산합니다. 

## 수비 이그노벨상 10개
1. 🏆 태클은 했지만...상 (Tackle Spam, Low Success)
2. 🏆 카드만 남겼다상 (Cards per Defensive Action)
3. 🏆 위험 지역 단골상 (Fouls in Defensive Third)
4. 🏆 클리어링 불안상 (Clearance → Concede Shot within 10s)
5. 🏆 블록은 했는데...상 (Block Attempts High, Low Success)
6. 🏆 인터셉트 헛발질상 (Interception Attempts High, Low Success)
7. 🏆 실점 직전 흔적상 (Pre-Goal Defensive Mistake Rate)
8. 🏆 듀얼은 많은데 지는 상 (Duel Volume High, Low Win)
9. 🏆 자기 진영 공 뺏김상 (Turnover Risk in Defensive Third)
10. 🏆 후반 집중력 붕괴상 (Second Half Defensive Drop)


In [1]:
%load_ext autoreload
%autoreload 2

import sys
from pathlib import Path

# Add project root to path
project_root = Path().resolve().parent
sys.path.insert(0, str(project_root))

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Import project modules
from src.data_loader import load_raw_events, load_match_info

# Set plotting style
plt.style.use("seaborn-v0_8")
sns.set_palette("husl")
plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False

print("✅ 라이브러리 로딩 완료")


✅ 라이브러리 로딩 완료


## 공통 전처리 (기준 정의)


In [2]:
# 데이터 로딩
print("데이터 로딩 중...")
raw = load_raw_events()
match_info = load_match_info()

print(f"이벤트 데이터: {len(raw):,}개")
print(f"경기 정보: {len(match_info):,}개")

# 시간 정렬 (이후 'n초 내 다음 이벤트' 같은 걸 위해)
raw = raw.sort_values(["game_id", "period_id", "time_seconds", "action_id"]).reset_index(drop=True)

# 성공/실패 플래그
raw["is_success"] = raw["result_name"].eq("Successful")
raw["is_fail"] = raw["result_name"].eq("Unsuccessful")

# 카드 플래그
CARD_SET = {"Yellow_Card", "Second_Yellow_Card", "Direct_Red_Card"}
raw["is_card"] = raw["result_name"].isin(CARD_SET)

# 수비 3rd (자기 진영) - start_x <= 35
raw["in_def_third"] = raw["start_x"] <= 35

print("\n✅ 전처리 완료")
print(f"성공 이벤트: {raw['is_success'].sum():,}개")
print(f"실패 이벤트: {raw['is_fail'].sum():,}개")
print(f"카드 이벤트: {raw['is_card'].sum():,}개")
print(f"수비 3rd 이벤트: {raw['in_def_third'].sum():,}개")


데이터 로딩 중...
이벤트 데이터: 579,306개
경기 정보: 198개

✅ 전처리 완료
성공 이벤트: 183,372개
실패 이벤트: 41,073개
카드 이벤트: 673개
수비 3rd 이벤트: 147,930개


## 1. 🏆 태클은 했지만...상 (Tackle Spam, Low Success)

**정의**: 태클을 많이 시도하지만 실패율이 높은 선수

**점수**: `tackle_fail_rate = 실패/시도` (높을수록 수상)


In [3]:
def calculate_tackle_fail_award(raw_df):
    """태클은 했지만...상 계산"""
    df = raw_df[raw_df["type_name"].eq("Tackle")].copy()
    
    score = df.groupby(["player_id", "player_name_ko", "team_name_ko"]).agg(
        tackle_attempt=("action_id", "count"),
        tackle_fail=("is_fail", "sum"),
    ).reset_index()
    
    score["tackle_fail_rate"] = score["tackle_fail"] / score["tackle_attempt"]
    score = score[score["tackle_attempt"] >= 5]  # 최소 5회 이상 시도한 선수만
    
    return score.sort_values("tackle_fail_rate", ascending=False)

tackle_award = calculate_tackle_fail_award(raw)

print("=" * 70)
print("🏆 태클은 했지만...상 TOP 10")
print("=" * 70)
print(tackle_award.head(10)[["player_name_ko", "team_name_ko", "tackle_attempt", "tackle_fail", "tackle_fail_rate"]])


🏆 태클은 했지만...상 TOP 10
    player_name_ko team_name_ko  tackle_attempt  tackle_fail  tackle_fail_rate
410           아라비제     울산 HD FC               6            6          1.000000
240            박재용    전북 현대 모터스               6            6          1.000000
311            박세민         대구FC              10           10          1.000000
336            이진용  김천 상무 프로축구단               6            6          1.000000
360            박찬용  김천 상무 프로축구단              17           16          0.941176
344            여홍규      제주SK FC              22           20          0.909091
397            김현욱    대전 하나 시티즌              11           10          0.909091
277            안혁주         광주FC              10            9          0.900000
146            박세진         대구FC              49           44          0.897959
68             이용래         대구FC               9            8          0.888889


## 2. 🏆 카드만 남겼다상 (Cards per Defensive Action)

**정의**: 수비 행동 대비 카드를 많이 받는 선수

**점수**: `card_per_def = 카드 / 수비행동` (높을수록 수상)


In [4]:
def calculate_card_per_def_award(raw_df):
    """카드만 남겼다상 계산"""
    DEF_ACTIONS = ["Tackle", "Duel", "Foul", "Interception", "Block", "Clearance", 
                   "Intervention", "Error", "Aerial Clearance"]
    df_def = raw_df[raw_df["type_name"].isin(DEF_ACTIONS)].copy()
    
    score = df_def.groupby(["player_id", "player_name_ko", "team_name_ko"]).agg(
        def_actions=("action_id", "count"),
        cards=("is_card", "sum"),
    ).reset_index()
    
    score["card_per_def"] = score["cards"] / score["def_actions"]
    score = score[score["def_actions"] >= 10]  # 최소 10회 이상 수비 행동
    
    return score.sort_values("card_per_def", ascending=False)

card_award = calculate_card_per_def_award(raw)

print("=" * 70)
print("🏆 카드만 남겼다상 TOP 10")
print("=" * 70)
print(card_award.head(10)[["player_name_ko", "team_name_ko", "def_actions", "cards", "card_per_def"]])


🏆 카드만 남겼다상 TOP 10
    player_name_ko team_name_ko  def_actions  cards  card_per_def
410            진준서         강원FC           15      2      0.133333
173            김경민         FC서울           15      2      0.133333
366            조인정      제주SK FC           10      1      0.100000
441            박주영      제주SK FC           12      1      0.083333
397            호날두         FC서울           12      1      0.083333
367            백승헌      제주SK FC           28      2      0.071429
25             이으뜸         광주FC           14      1      0.071429
34             백종범         FC서울           59      4      0.067797
421            하지치         강원FC           15      1      0.066667
440            강현무         FC서울           51      3      0.058824


## 3. 🏆 위험 지역 단골상 (Fouls in Defensive Third)

**정의**: 수비 3rd에서 파울을 많이 하는 선수

**점수**: `danger_foul_ratio = 수비3rd파울/전체파울` (높을수록 수상)


In [5]:
def calculate_danger_foul_award(raw_df):
    """위험 지역 단골상 계산"""
    FOUL_TYPES = ["Foul", "Handball_Foul", "Hit"]
    df = raw_df[raw_df["type_name"].isin(FOUL_TYPES)].copy()
    
    g = df.groupby(["player_id", "player_name_ko", "team_name_ko"]).agg(
        fouls=("action_id", "count"),
        danger_fouls=("in_def_third", "sum"),
    ).reset_index()
    
    g["danger_foul_ratio"] = g["danger_fouls"] / g["fouls"]
    g = g[g["fouls"] >= 3]  # 최소 3회 이상 파울
    
    return g.sort_values("danger_foul_ratio", ascending=False)

foul_award = calculate_danger_foul_award(raw)

print("=" * 70)
print("🏆 위험 지역 단골상 TOP 10")
print("=" * 70)
print(foul_award.head(10)[["player_name_ko", "team_name_ko", "fouls", "danger_fouls", "danger_foul_ratio"]])


🏆 위험 지역 단골상 TOP 10
    player_name_ko team_name_ko  fouls  danger_fouls  danger_foul_ratio
397            강현무         FC서울      3             3           1.000000
55             안창민         대구FC      5             4           0.800000
211            아사니         광주FC      8             6           0.750000
363            박정인    대전 하나 시티즌      3             2           0.666667
339           시게히로         FC서울      3             2           0.666667
82             정동호         수원FC      3             2           0.666667
366            안재준      포항 스틸러스      3             2           0.666667
113            하승운         광주FC      3             2           0.666667
282            백민규     인천 유나이티드      3             2           0.666667
145            김영준         대구FC      6             4           0.666667


## 4. 🏆 클리어링 불안상 (Clearance → Concede Shot within 10s)

**정의**: 클리어링 후 10초 내 상대 슈팅을 허용하는 선수

**점수**: `clearance_panic_rate = (10초내상대슛)/클리어링` (높을수록 수상)


In [11]:
def calculate_clearance_panic_award(raw_df):
    """클리어링 불안상 계산"""
    clr = raw_df[raw_df["type_name"].isin(["Clearance", "Aerial Clearance"])].copy()
    shots = raw_df[raw_df["type_name"].eq("Shot")].copy()
    
    if len(clr) == 0 or len(shots) == 0:
        return pd.DataFrame()
    
    # 각 경기/하프별로 처리하여 merge_asof 정렬 문제 해결
    results = []
    
    for (game_id, period_id), clr_group in clr.groupby(["game_id", "period_id"]):
        shots_group = shots[(shots["game_id"] == game_id) & (shots["period_id"] == period_id)].copy()
        
        if len(shots_group) == 0:
            continue
        
        # 각 그룹별로 정렬
        clr_group = clr_group.sort_values("time_seconds").reset_index(drop=True)
        shots_group = shots_group.sort_values("time_seconds").reset_index(drop=True)
        
        # merge_asof로 클리어링 이후 슈팅 찾기
        try:
            m = pd.merge_asof(
                clr_group, shots_group,
                on="time_seconds",
                direction="forward",
                suffixes=("_clr", "_shot"),
                allow_exact_matches=True
            )
            
            # NaN 값 처리 (매칭되지 않은 경우)
            m = m.dropna(subset=["time_seconds_shot", "team_id_shot"])
            
            if len(m) == 0:
                continue
            
            # 10초 이내 + 상대팀 슈팅
            m["time_diff"] = m["time_seconds_shot"] - m["time_seconds_clr"]
            m["shot_within_10s"] = (
                (m["time_diff"] <= 10) & 
                (m["time_diff"] >= 0) & 
                (m["team_id_shot"] != m["team_id_clr"])
            )
            
            results.append(m)
        except (ValueError, KeyError) as e:
            # 정렬 문제나 컬럼 문제가 있으면 스킵
            continue
    
    if len(results) == 0:
        return pd.DataFrame()
    
    # 모든 결과 합치기
    m = pd.concat(results, ignore_index=True)
    
    # 필요한 컬럼 확인
    required_cols = ["player_id_clr", "player_name_ko_clr", "team_name_ko_clr", "action_id_clr", "shot_within_10s"]
    missing_cols = [col for col in required_cols if col not in m.columns]
    if missing_cols:
        print(f"경고: 필요한 컬럼이 없습니다: {missing_cols}")
        return pd.DataFrame()
    
    score = m.groupby(["player_id_clr", "player_name_ko_clr", "team_name_ko_clr"]).agg(
        clearance=("action_id_clr", "count"),
        concede_shot10=("shot_within_10s", "sum"),
    ).reset_index()
    
    score.columns = ["player_id", "player_name_ko", "team_name_ko", "clearance", "concede_shot10"]
    score["clearance_panic_rate"] = score["concede_shot10"] / score["clearance"]
    score = score[score["clearance"] >= 5]  # 최소 5회 이상 클리어링
    
    return score.sort_values("clearance_panic_rate", ascending=False)

clearance_award = calculate_clearance_panic_award(raw)

print("=" * 70)
print("🏆 클리어링 불안상 TOP 10")
print("=" * 70)
if len(clearance_award) > 0:
    print(clearance_award.head(10)[["player_name_ko", "team_name_ko", "clearance", "concede_shot10", "clearance_panic_rate"]])
else:
    print("데이터가 부족합니다.")


🏆 클리어링 불안상 TOP 10
데이터가 부족합니다.


## 5. 🏆 블록은 했는데...상 (Block Attempts High, Low Success)

**정의**: 블록을 많이 시도하지만 실패율이 높은 선수

**점수**: `block_fail_rate = 실패/시도` (높을수록 수상)


In [12]:
def calculate_block_fail_award(raw_df):
    """블록은 했는데...상 계산"""
    df = raw_df[raw_df["type_name"].eq("Block")].copy()
    
    if len(df) == 0:
        return pd.DataFrame()
    
    g = df.groupby(["player_id", "player_name_ko", "team_name_ko"]).agg(
        block_attempt=("action_id", "count"),
        block_fail=("is_fail", "sum"),
    ).reset_index()
    
    g["block_fail_rate"] = g["block_fail"] / g["block_attempt"]
    g = g[g["block_attempt"] >= 3]  # 최소 3회 이상 시도
    
    return g.sort_values("block_fail_rate", ascending=False)

block_award = calculate_block_fail_award(raw)

print("=" * 70)
print("🏆 블록은 했는데...상 TOP 10")
print("=" * 70)
if len(block_award) > 0:
    print(block_award.head(10)[["player_name_ko", "team_name_ko", "block_attempt", "block_fail", "block_fail_rate"]])
else:
    print("데이터가 부족합니다.")


🏆 블록은 했는데...상 TOP 10
    player_name_ko team_name_ko  block_attempt  block_fail  block_fail_rate
398           아라비제     울산 HD FC              3           0              0.0
0              김동민     인천 유나이티드             24           0              0.0
1              정동윤     인천 유나이티드             40           0              0.0
2              김보섭     인천 유나이티드             12           0              0.0
3              무고사     인천 유나이티드             17           0              0.0
4              김진수    전북 현대 모터스             39           0              0.0
5              최철순    전북 현대 모터스              5           0              0.0
6              홍정호    전북 현대 모터스             25           0              0.0
7              한교원    전북 현대 모터스             10           0              0.0
377          코바체비치         강원FC              6           0              0.0


## 6. 🏆 인터셉트 헛발질상 (Interception Attempts High, Low Success)

**정의**: 인터셉트를 많이 시도하지만 실패율이 높은 선수

**점수**: `interception_fail_rate = 실패/시도` (높을수록 수상)


In [13]:
def calculate_interception_fail_award(raw_df):
    """인터셉트 헛발질상 계산"""
    df = raw_df[raw_df["type_name"].eq("Interception")].copy()
    
    if len(df) == 0:
        return pd.DataFrame()
    
    g = df.groupby(["player_id", "player_name_ko", "team_name_ko"]).agg(
        int_attempt=("action_id", "count"),
        int_fail=("is_fail", "sum"),
    ).reset_index()
    
    g["int_fail_rate"] = g["int_fail"] / g["int_attempt"]
    g = g[g["int_attempt"] >= 5]  # 최소 5회 이상 시도
    
    return g.sort_values("int_fail_rate", ascending=False)

interception_award = calculate_interception_fail_award(raw)

print("=" * 70)
print("🏆 인터셉트 헛발질상 TOP 10")
print("=" * 70)
if len(interception_award) > 0:
    print(interception_award.head(10)[["player_name_ko", "team_name_ko", "int_attempt", "int_fail", "int_fail_rate"]])
else:
    print("데이터가 부족합니다.")


🏆 인터셉트 헛발질상 TOP 10
    player_name_ko team_name_ko  int_attempt  int_fail  int_fail_rate
419           아라비제     울산 HD FC            6         0            0.0
0              김동민     인천 유나이티드           59         0            0.0
1              정동윤     인천 유나이티드           72         0            0.0
2              김보섭     인천 유나이티드            8         0            0.0
3              무고사     인천 유나이티드           28         0            0.0
4              김진수    전북 현대 모터스           71         0            0.0
5              최철순    전북 현대 모터스           15         0            0.0
6              홍정호    전북 현대 모터스           82         0            0.0
7              한교원    전북 현대 모터스           16         0            0.0
8              진성욱      제주SK FC           20         0            0.0


## 7. 🏆 실점 직전 흔적상 (Pre-Goal Defensive Mistake Rate)

**정의**: 실점 직전 10초 내 수비 실패/에러가 자주 등장하는 선수

**점수**: `pre_goal_mistake_rate = 실점직전실패/전체수비행동` (높을수록 수상)


In [15]:
def calculate_pre_goal_mistake_award(raw_df, match_info_df):
    """실점 직전 흔적상 계산"""
    # 골 이벤트 찾기
    goals = raw_df[raw_df["type_name"].eq("Goal")].copy()
    
    if len(goals) == 0:
        return pd.DataFrame()
    
    # 실점팀 ID 계산
    goals = goals.merge(match_info_df[["game_id", "home_team_id", "away_team_id"]], 
                       on="game_id", how="left")
    goals["concede_team_id"] = goals.apply(
        lambda r: r["away_team_id"] if r["team_id"] == r["home_team_id"] else r["home_team_id"],
        axis=1
    )
    
    # 수비 실패/에러 이벤트
    MISTAKE_TYPES = ["Tackle", "Duel", "Interception", "Block", "Clearance", 
                     "Aerial Clearance", "Error", "Intervention"]
    cand = raw_df[raw_df["type_name"].isin(MISTAKE_TYPES)].copy()
    
    if len(cand) == 0:
        return pd.DataFrame()
    
    # 골 직전 10초 내 같은 경기/하프 & 실점팀 수비 이벤트
    # 각 경기/하프별로 처리하여 merge_asof 정렬 문제 해결
    results = []
    
    for (game_id, period_id), goals_group in goals.groupby(["game_id", "period_id"]):
        cand_group = cand[(cand["game_id"] == game_id) & (cand["period_id"] == period_id)].copy()
        
        if len(cand_group) == 0:
            continue
        
        # 각 그룹별로 정렬
        goals_group = goals_group.sort_values("time_seconds").reset_index(drop=True)
        cand_group = cand_group.sort_values("time_seconds").reset_index(drop=True)
        
        # merge_asof로 골 직전 이벤트 찾기 (backward)
        try:
            m_group = pd.merge_asof(
                goals_group, cand_group,
                on="time_seconds",
                direction="backward",
                suffixes=("_goal", "_def"),
                allow_exact_matches=True
            )
            
            # NaN 값 처리 (매칭되지 않은 경우)
            # time_seconds는 left에 있으므로 _goal 접미사가 붙지 않을 수 있음
            # 컬럼명 확인 및 처리
            time_col_goal = "time_seconds_goal" if "time_seconds_goal" in m_group.columns else "time_seconds"
            time_col_def = "time_seconds_def" if "time_seconds_def" in m_group.columns else None
            
            if time_col_def is None:
                continue
            
            # NaN 값이 있는 행 제거
            m_group = m_group.dropna(subset=[time_col_def, "team_id_def"])
            
            if len(m_group) == 0:
                continue
            
            # time_diff 계산
            m_group["time_diff"] = m_group[time_col_goal] - m_group[time_col_def]
            m_group["within_10s"] = (m_group["time_diff"] <= 10) & (m_group["time_diff"] >= 0)
            m_group["is_concede_team"] = (m_group["team_id_def"] == m_group["concede_team_id"])
            m_group["is_mistake"] = m_group["type_name_def"].eq("Error") | m_group["result_name_def"].eq("Unsuccessful")
            
            results.append(m_group)
        except (ValueError, KeyError) as e:
            # 정렬 문제나 컬럼 문제가 있으면 스킵
            continue
    
    if len(results) == 0:
        return pd.DataFrame()
    
    # 모든 결과 합치기
    m = pd.concat(results, ignore_index=True)
    
    # 필요한 컬럼 확인
    required_cols = ["within_10s", "is_concede_team", "is_mistake", "player_id_def", "player_name_ko_def", "team_name_ko_def"]
    missing_cols = [col for col in required_cols if col not in m.columns]
    if missing_cols:
        print(f"경고: 필요한 컬럼이 없습니다: {missing_cols}")
        return pd.DataFrame()
    
    # 필터링
    pre = m[(m["within_10s"]) & (m["is_concede_team"]) & (m["is_mistake"])]
    
    # 분모: 전체 수비행동
    DEF_ACTIONS = ["Tackle", "Duel", "Foul", "Interception", "Block", "Clearance", 
                   "Intervention", "Error", "Aerial Clearance"]
    df_def = raw_df[raw_df["type_name"].isin(DEF_ACTIONS)]
    den = df_def.groupby(["player_id", "player_name_ko", "team_name_ko"]).size().reset_index(name="def_actions")
    
    # 분자: 실점 직전 실패
    if len(pre) > 0:
        num = pre.groupby(["player_id_def", "player_name_ko_def", "team_name_ko_def"]).size().reset_index(name="pre_goal_mistakes")
        num.columns = ["player_id", "player_name_ko", "team_name_ko", "pre_goal_mistakes"]
        
        score = den.merge(num, on=["player_id", "player_name_ko", "team_name_ko"], how="left").fillna(0)
    else:
        score = den.copy()
        score["pre_goal_mistakes"] = 0
    
    score["pre_goal_mistake_rate"] = score["pre_goal_mistakes"] / score["def_actions"]
    score = score[score["def_actions"] >= 20]  # 최소 20회 이상 수비 행동
    
    return score.sort_values("pre_goal_mistake_rate", ascending=False)

pre_goal_award = calculate_pre_goal_mistake_award(raw, match_info)

print("=" * 70)
print("🏆 실점 직전 흔적상 TOP 10")
print("=" * 70)
if len(pre_goal_award) > 0:
    print(pre_goal_award.head(10)[["player_name_ko", "team_name_ko", "def_actions", "pre_goal_mistakes", "pre_goal_mistake_rate"]])
else:
    print("데이터가 부족합니다.")


🏆 실점 직전 흔적상 TOP 10
데이터가 부족합니다.


In [16]:
def calculate_duel_fail_award(raw_df):
    """듀얼은 많은데 지는 상 계산"""
    df = raw_df[raw_df["type_name"].eq("Duel")].copy()
    
    if len(df) == 0:
        return pd.DataFrame()
    
    g = df.groupby(["player_id", "player_name_ko", "team_name_ko"]).agg(
        duel_attempt=("action_id", "count"),
        duel_fail=("is_fail", "sum"),
    ).reset_index()
    
    g["duel_fail_rate"] = g["duel_fail"] / g["duel_attempt"]
    g = g[g["duel_attempt"] >= 10]  # 최소 10회 이상 시도
    
    return g.sort_values("duel_fail_rate", ascending=False)

duel_award = calculate_duel_fail_award(raw)

print("=" * 70)
print("🏆 듀얼은 많은데 지는 상 TOP 10")
print("=" * 70)
if len(duel_award) > 0:
    print(duel_award.head(10)[["player_name_ko", "team_name_ko", "duel_attempt", "duel_fail", "duel_fail_rate"]])
else:
    print("데이터가 부족합니다.")


🏆 듀얼은 많은데 지는 상 TOP 10
    player_name_ko team_name_ko  duel_attempt  duel_fail  duel_fail_rate
371            이상민    대전 하나 시티즌            18          1        0.055556
174           윤빛가람         수원FC            41          1        0.024390
46             최규백         수원FC            71          1        0.014085
164         유리 조나탄      제주SK FC           188          1        0.005319
331            이탈로      제주SK FC            98          0        0.000000
23             안영규         광주FC            67          0        0.000000
0              김동민     인천 유나이티드            33          0        0.000000
332            김정민      제주SK FC            17          0        0.000000
334            강상우         FC서울            45          0        0.000000
337            정한민         강원FC            17          0        0.000000


## 9. 🏆 자기 진영 공 뺏김상 (Turnover Risk in Defensive Third)

**정의**: 수비 3rd에서 패스/캐리 실패율이 높은 선수

**점수**: `def_third_turnover_rate = 실패/시도` (높을수록 수상)


In [17]:
def calculate_def_third_turnover_award(raw_df):
    """자기 진영 공 뺏김상 계산"""
    df = raw_df[
        raw_df["type_name"].isin(["Pass", "Carry"]) & 
        raw_df["in_def_third"]
    ].copy()
    
    if len(df) == 0:
        return pd.DataFrame()
    
    g = df.groupby(["player_id", "player_name_ko", "team_name_ko"]).agg(
        attempts=("action_id", "count"),
        fails=("is_fail", "sum"),
    ).reset_index()
    
    g["def_third_turnover_rate"] = g["fails"] / g["attempts"]
    g = g[g["attempts"] >= 10]  # 최소 10회 이상 시도
    
    return g.sort_values("def_third_turnover_rate", ascending=False)

turnover_award = calculate_def_third_turnover_award(raw)

print("=" * 70)
print("🏆 자기 진영 공 뺏김상 TOP 10")
print("=" * 70)
if len(turnover_award) > 0:
    print(turnover_award.head(10)[["player_name_ko", "team_name_ko", "attempts", "fails", "def_third_turnover_rate"]])
else:
    print("데이터가 부족합니다.")


🏆 자기 진영 공 뺏김상 TOP 10
    player_name_ko team_name_ko  attempts  fails  def_third_turnover_rate
408            하지치         강원FC        12      7                 0.583333
105            이준호    전북 현대 모터스        11      4                 0.363636
420            홍재석      제주SK FC        12      4                 0.333333
131            송창석    대전 하나 시티즌        16      5                 0.312500
356            백승헌      제주SK FC        15      4                 0.266667
245             구텍    대전 하나 시티즌        27      7                 0.259259
99             공민현    대전 하나 시티즌        36      9                 0.250000
249            박재용    전북 현대 모터스        28      7                 0.250000
112            오승훈         대구FC       273     67                 0.245421
255            조성준      포항 스틸러스        13      3                 0.230769


## 10. 🏆 후반 집중력 붕괴상 (Second Half Defensive Drop)

**정의**: 후반 수비 실패율이 전반보다 크게 증가한 선수

**점수**: `second_half_drop = 후반실패율 - 전반실패율` (높을수록 수상)


In [18]:
def calculate_second_half_drop_award(raw_df):
    """후반 집중력 붕괴상 계산"""
    DEF_ACTIONS = ["Tackle", "Duel", "Foul", "Interception", "Block", "Clearance", 
                   "Intervention", "Error", "Aerial Clearance"]
    df = raw_df[raw_df["type_name"].isin(DEF_ACTIONS)].copy()
    
    if len(df) == 0:
        return pd.DataFrame()
    
    df["is_def_fail"] = df["result_name"].eq("Unsuccessful") | df["type_name"].eq("Error")
    
    # 전반/후반 실패율
    g = df.groupby(["player_id", "player_name_ko", "team_name_ko", "period_id"]).agg(
        def_actions=("action_id", "count"),
        def_fails=("is_def_fail", "sum"),
    ).reset_index()
    
    g["fail_rate"] = g["def_fails"] / g["def_actions"]
    
    # period_id=1 vs 2 비교
    p = g.pivot_table(
        index=["player_id", "player_name_ko", "team_name_ko"],
        columns="period_id",
        values="fail_rate",
        fill_value=0
    ).reset_index()
    
    # period_id 컬럼명 정리
    period_cols = [col for col in p.columns if col in [1, 2]]
    if len(period_cols) == 2:
        p["first_half_rate"] = p[1]
        p["second_half_rate"] = p[2]
        p["second_half_drop"] = p["second_half_rate"] - p["first_half_rate"]
        
        # 최소 행동 수 필터링
        total_actions = df.groupby(["player_id", "player_name_ko", "team_name_ko"]).size().reset_index(name="total_actions")
        p = p.merge(total_actions, on=["player_id", "player_name_ko", "team_name_ko"])
        p = p[p["total_actions"] >= 20]
        
        return p.sort_values("second_half_drop", ascending=False)
    else:
        return pd.DataFrame()

second_half_award = calculate_second_half_drop_award(raw)

print("=" * 70)
print("🏆 후반 집중력 붕괴상 TOP 10")
print("=" * 70)
if len(second_half_award) > 0:
    cols = ["player_name_ko", "team_name_ko", "first_half_rate", "second_half_rate", "second_half_drop"]
    print(second_half_award.head(10)[cols])
else:
    print("데이터가 부족합니다.")


🏆 후반 집중력 붕괴상 TOP 10
    player_name_ko team_name_ko  first_half_rate  second_half_rate  \
383            강주혁         FC서울         0.000000          0.354839   
228            이영준  김천 상무 프로축구단         0.066667          0.400000   
74             이용래         대구FC         0.000000          0.321429   
368            여홍규      제주SK FC         0.320988          0.571429   
201            이광혁         수원FC         0.000000          0.238095   
170            이승준         FC서울         0.000000          0.227273   
71             신상은    대전 하나 시티즌         0.139535          0.333333   
156            갈레고         강원FC         0.142857          0.333333   
404            카이나      제주SK FC         0.188406          0.375000   
90             강윤구     울산 HD FC         0.148649          0.333333   

     second_half_drop  
383          0.354839  
228          0.333333  
74           0.321429  
368          0.250441  
201          0.238095  
170          0.227273  
71           0.193798  
156          0.19

## 📊 종합 결과: 수비 이그노벨상 TOP 10

각 상의 TOP 10 선수를 종합하여 최종 수상자를 결정합니다.


In [19]:
def generate_ignobel_summary():
    """수비 이그노벨상 종합 리포트 생성"""
    awards = {
        "태클은 했지만...상": tackle_award.head(10) if len(tackle_award) > 0 else pd.DataFrame(),
        "카드만 남겼다상": card_award.head(10) if len(card_award) > 0 else pd.DataFrame(),
        "위험 지역 단골상": foul_award.head(10) if len(foul_award) > 0 else pd.DataFrame(),
        "클리어링 불안상": clearance_award.head(10) if len(clearance_award) > 0 else pd.DataFrame(),
        "블록은 했는데...상": block_award.head(10) if len(block_award) > 0 else pd.DataFrame(),
        "인터셉트 헛발질상": interception_award.head(10) if len(interception_award) > 0 else pd.DataFrame(),
        "실점 직전 흔적상": pre_goal_award.head(10) if len(pre_goal_award) > 0 else pd.DataFrame(),
        "듀얼은 많은데 지는 상": duel_award.head(10) if len(duel_award) > 0 else pd.DataFrame(),
        "자기 진영 공 뺏김상": turnover_award.head(10) if len(turnover_award) > 0 else pd.DataFrame(),
        "후반 집중력 붕괴상": second_half_award.head(10) if len(second_half_award) > 0 else pd.DataFrame(),
    }
    
    return awards

# 종합 리포트 생성
all_awards = generate_ignobel_summary()

print("=" * 70)
print("📊 수비 이그노벨상 종합 리포트")
print("=" * 70)

for award_name, award_df in all_awards.items():
    if len(award_df) > 0:
        print(f"\n🏆 {award_name}")
        print("-" * 70)
        if "player_name_ko" in award_df.columns:
            top3 = award_df.head(3)
            for idx, row in top3.iterrows():
                print(f"  {idx+1}. {row['player_name_ko']} ({row.get('team_name_ko', 'N/A')})")
        else:
            print("  데이터 없음")


📊 수비 이그노벨상 종합 리포트

🏆 태클은 했지만...상
----------------------------------------------------------------------
  411. 아라비제 (울산 HD FC)
  241. 박재용 (전북 현대 모터스)
  312. 박세민 (대구FC)

🏆 카드만 남겼다상
----------------------------------------------------------------------
  411. 진준서 (강원FC)
  174. 김경민 (FC서울)
  367. 조인정 (제주SK FC)

🏆 위험 지역 단골상
----------------------------------------------------------------------
  398. 강현무 (FC서울)
  56. 안창민 (대구FC)
  212. 아사니 (광주FC)

🏆 블록은 했는데...상
----------------------------------------------------------------------
  399. 아라비제 (울산 HD FC)
  1. 김동민 (인천 유나이티드)
  2. 정동윤 (인천 유나이티드)

🏆 인터셉트 헛발질상
----------------------------------------------------------------------
  420. 아라비제 (울산 HD FC)
  1. 김동민 (인천 유나이티드)
  2. 정동윤 (인천 유나이티드)

🏆 듀얼은 많은데 지는 상
----------------------------------------------------------------------
  372. 이상민 (대전 하나 시티즌)
  175. 윤빛가람 (수원FC)
  47. 최규백 (수원FC)

🏆 자기 진영 공 뺏김상
----------------------------------------------------------------------
  409. 하지치 (강원FC)
  106. 이준

## 🎨 이그노벨상 카드 템플릿 생성

각 수상자에 대한 재미있는 카드 문구를 자동 생성합니다.


In [20]:
def generate_ignobel_card(award_name, player_name, team_name, stats_dict):
    """이그노벨상 카드 템플릿 생성"""
    
    templates = {
        "태클은 했지만...상": {
            "title": "🏆 태클은 했지만...상",
            "subtitle": "Tackle Spam, Low Success Award",
            "message": f"{player_name} 선수는 태클을 {stats_dict.get('tackle_attempt', 0):.0f}회 시도했지만, "
                      f"실패율이 {stats_dict.get('tackle_fail_rate', 0)*100:.1f}%로 매우 높습니다. "
                      f"의욕은 좋지만... 정확도가 문제네요! 😅"
        },
        "카드만 남겼다상": {
            "title": "🏆 카드만 남겼다상",
            "subtitle": "Cards per Defensive Action Award",
            "message": f"{player_name} 선수는 수비 행동 {stats_dict.get('def_actions', 0):.0f}회당 "
                      f"카드를 {stats_dict.get('card_per_def', 0)*100:.2f}% 받았습니다. "
                      f"수비는 했지만... 심판님 눈에 너무 많이 띄셨나봅니다! 🟨🟥"
        },
        "위험 지역 단골상": {
            "title": "🏆 위험 지역 단골상",
            "subtitle": "Fouls in Defensive Third Award",
            "message": f"{player_name} 선수는 전체 파울 {stats_dict.get('fouls', 0):.0f}회 중 "
                      f"{stats_dict.get('danger_foul_ratio', 0)*100:.1f}%를 수비 3rd에서 범했습니다. "
                      f"위험한 지역에서의 실수는... 위험합니다! ⚠️"
        },
        "클리어링 불안상": {
            "title": "🏆 클리어링 불안상",
            "subtitle": "Clearance → Concede Shot within 10s Award",
            "message": f"{player_name} 선수는 클리어링 후 10초 내 상대 슈팅을 "
                      f"{stats_dict.get('clearance_panic_rate', 0)*100:.1f}% 확률로 허용했습니다. "
                      f"클리어링은 했지만... 바로 다시 위험해졌네요! 😰"
        },
        "블록은 했는데...상": {
            "title": "🏆 블록은 했는데...상",
            "subtitle": "Block Attempts High, Low Success Award",
            "message": f"{player_name} 선수는 블록을 {stats_dict.get('block_attempt', 0):.0f}회 시도했지만, "
                      f"실패율이 {stats_dict.get('block_fail_rate', 0)*100:.1f}%입니다. "
                      f"블록 시도는 많지만... 성공률이 아쉽네요! 🛡️"
        },
        "인터셉트 헛발질상": {
            "title": "🏆 인터셉트 헛발질상",
            "subtitle": "Interception Attempts High, Low Success Award",
            "message": f"{player_name} 선수는 인터셉트를 {stats_dict.get('int_attempt', 0):.0f}회 시도했지만, "
                      f"실패율이 {stats_dict.get('int_fail_rate', 0)*100:.1f}%입니다. "
                      f"공을 가로채려는 의지는 좋지만... 정확도가 문제입니다! 🎯"
        },
        "실점 직전 흔적상": {
            "title": "🏆 실점 직전 흔적상",
            "subtitle": "Pre-Goal Defensive Mistake Rate Award",
            "message": f"{player_name} 선수는 실점 직전 10초 내 수비 실패가 "
                      f"전체 수비 행동의 {stats_dict.get('pre_goal_mistake_rate', 0)*100:.2f}%를 차지합니다. "
                      f"실점 직전의 실수는... 치명적입니다! 💥"
        },
        "듀얼은 많은데 지는 상": {
            "title": "🏆 듀얼은 많은데 지는 상",
            "subtitle": "Duel Volume High, Low Win Award",
            "message": f"{player_name} 선수는 듀얼을 {stats_dict.get('duel_attempt', 0):.0f}회 시도했지만, "
                      f"실패율이 {stats_dict.get('duel_fail_rate', 0)*100:.1f}%입니다. "
                      f"많이 싸우지만... 많이 지네요! ⚔️"
        },
        "자기 진영 공 뺏김상": {
            "title": "🏆 자기 진영 공 뺏김상",
            "subtitle": "Turnover Risk in Defensive Third Award",
            "message": f"{player_name} 선수는 수비 3rd에서 패스/캐리 실패율이 "
                      f"{stats_dict.get('def_third_turnover_rate', 0)*100:.1f}%입니다. "
                      f"자기 진영에서 공을 빼앗기는 건... 위험합니다! 🚨"
        },
        "후반 집중력 붕괴상": {
            "title": "🏆 후반 집중력 붕괴상",
            "subtitle": "Second Half Defensive Drop Award",
            "message": f"{player_name} 선수는 전반 실패율 {stats_dict.get('first_half_rate', 0)*100:.1f}%에서 "
                      f"후반 {stats_dict.get('second_half_rate', 0)*100:.1f}%로 "
                      f"{stats_dict.get('second_half_drop', 0)*100:.1f}%p 증가했습니다. "
                      f"후반 집중력이... 떨어지네요! 📉"
        }
    }
    
    return templates.get(award_name, {
        "title": f"🏆 {award_name}",
        "subtitle": "Award",
        "message": f"{player_name} 선수가 수상했습니다!"
    })

# 각 상의 TOP 3에 대해 카드 생성
print("=" * 70)
print("🎨 수비 이그노벨상 카드")
print("=" * 70)

for award_name, award_df in all_awards.items():
    if len(award_df) > 0:
        top_player = award_df.iloc[0]
        player_name = top_player.get('player_name_ko', 'Unknown')
        team_name = top_player.get('team_name_ko', 'Unknown')
        
        # 통계 딕셔너리 생성
        stats = top_player.to_dict()
        
        card = generate_ignobel_card(award_name, player_name, team_name, stats)
        
        print(f"\n{card['title']}")
        print(f"{card['subtitle']}")
        print(f"\n수상자: {player_name} ({team_name})")
        print(f"\n{card['message']}")
        print("-" * 70)


🎨 수비 이그노벨상 카드

🏆 태클은 했지만...상
Tackle Spam, Low Success Award

수상자: 아라비제 (울산 HD FC)

아라비제 선수는 태클을 6회 시도했지만, 실패율이 100.0%로 매우 높습니다. 의욕은 좋지만... 정확도가 문제네요! 😅
----------------------------------------------------------------------

🏆 카드만 남겼다상
Cards per Defensive Action Award

수상자: 진준서 (강원FC)

진준서 선수는 수비 행동 15회당 카드를 13.33% 받았습니다. 수비는 했지만... 심판님 눈에 너무 많이 띄셨나봅니다! 🟨🟥
----------------------------------------------------------------------

🏆 위험 지역 단골상
Fouls in Defensive Third Award

수상자: 강현무 (FC서울)

강현무 선수는 전체 파울 3회 중 100.0%를 수비 3rd에서 범했습니다. 위험한 지역에서의 실수는... 위험합니다! ⚠️
----------------------------------------------------------------------

🏆 블록은 했는데...상
Block Attempts High, Low Success Award

수상자: 아라비제 (울산 HD FC)

아라비제 선수는 블록을 3회 시도했지만, 실패율이 0.0%입니다. 블록 시도는 많지만... 성공률이 아쉽네요! 🛡️
----------------------------------------------------------------------

🏆 인터셉트 헛발질상
Interception Attempts High, Low Success Award

수상자: 아라비제 (울산 HD FC)

아라비제 선수는 인터셉트를 6회 시도했지만, 실패율이 0.0%입니다. 공을 가로채려는 의지는 좋지만... 정확도가 문제입니다! 🎯
